In [2]:
import tensorflow as tf
from tensorflow import keras

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.boston_housing.load_data()

65536/57026 [==================================] - 0s 0us/step


In [3]:
print(X_train)

[[1.23247e+00 0.00000e+00 8.14000e+00 ... 2.10000e+01 3.96900e+02
  1.87200e+01]
 [2.17700e-02 8.25000e+01 2.03000e+00 ... 1.47000e+01 3.95380e+02
  3.11000e+00]
 [4.89822e+00 0.00000e+00 1.81000e+01 ... 2.02000e+01 3.75520e+02
  3.26000e+00]
 ...
 [3.46600e-02 3.50000e+01 6.06000e+00 ... 1.69000e+01 3.62250e+02
  7.83000e+00]
 [2.14918e+00 0.00000e+00 1.95800e+01 ... 1.47000e+01 2.61950e+02
  1.57900e+01]
 [1.43900e-02 6.00000e+01 2.93000e+00 ... 1.56000e+01 3.76700e+02
  4.38000e+00]]


In [4]:
def build_model(n_hidden=1, n_neurons=25, optimizer="sgd", learning_rate=1e5, momentum=0): 
    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input(shape=(X_train[0].shape)))
    for x in range(0, n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation='relu'))
    if optimizer=="sgd":
        optimizer = keras.optimizers.SGD(
        learning_rate=learning_rate, momentum=0.0, nesterov=False, name="SGD")
    elif optimizer=="adam":
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer=="nesterov":
        optimizer = keras.optimizers.SGD(
        learning_rate=learning_rate, momentum=0.0, nesterov=True, name="SGD")
    elif optimizer=="momentum":
        optimizer = keras.optimizers.SGD(
        learning_rate=learning_rate, momentum=momentum, nesterov=False, name="SGD")
        
    model.add(tf.keras.layers.Dense(1))
    
    model.compile(loss="MSE",
              optimizer=optimizer,metrics=["MAE"])
   

    return model

In [5]:
import os
root_logdir = os.path.join(os.curdir, "tb_logs")
def get_run_logdir(name, value): 
    import time
    ts = int(time.time())
    run_id = str(ts)+"_"+str(name)+"_"+str(value) 
    return os.path.join(root_logdir, run_id)

In [6]:
es = tf.keras.callbacks.EarlyStopping(patience=10, min_delta=1.0,verbose=1)

In [7]:
import numpy as np
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [8]:
lr = []
for x in [10e-6,10e-5,10e-4]:
    model = build_model(1,25,"sgd",x)
    tensorboard_cb = tf.keras.callbacks.TensorBoard(get_run_logdir("lr",x))
    model.fit(X_train, y_train, epochs=100,  callbacks=[tensorboard_cb,es], validation_split=0.1)
    ev = model.evaluate(X_test, y_test)
    lr.append((x,ev[0],ev[1]))
import pickle
pickle.dump(lr, open("lr.pkl", "wb"))

Epoch 1/100
12/12 [==============================] - 1s 17ms/step - loss: 117101.7812 - MAE: 123.7935 - val_loss: 108.1399 - val_MAE: 8.5337
Epoch 2/100
12/12 [==============================] - 0s 6ms/step - loss: 163.8253 - MAE: 9.5988 - val_loss: 68.6999 - val_MAE: 6.7683
Epoch 3/100
12/12 [==============================] - 0s 5ms/step - loss: 113.2871 - MAE: 7.8936 - val_loss: 49.6265 - val_MAE: 5.5180
Epoch 4/100
12/12 [==============================] - 0s 3ms/step - loss: 97.5250 - MAE: 7.2386 - val_loss: 44.5197 - val_MAE: 5.2958
Epoch 5/100
12/12 [==============================] - 0s 5ms/step - loss: 91.5391 - MAE: 7.0076 - val_loss: 42.7563 - val_MAE: 5.2981
Epoch 6/100
12/12 [==============================] - 0s 6ms/step - loss: 87.4378 - MAE: 7.0010 - val_loss: 42.5495 - val_MAE: 5.0830
Epoch 7/100
12/12 [==============================] - 0s 5ms/step - loss: 86.9827 - MAE: 6.7954 - val_loss: 41.1674 - val_MAE: 5.1446
Epoch 8/100
12/12 [==============================] - 0s 5ms

In [9]:
hl = []
for x in [0,1,2,3]:
    model = build_model(x,25,"sgd",10e-5)
    tensorboard_cb = tf.keras.callbacks.TensorBoard(get_run_logdir("hl",x))
    model.fit(X_train, y_train, epochs=100,  callbacks=[tensorboard_cb,es], validation_split=0.1) 
    ev = model.evaluate(X_test, y_test)
    hl.append((x,ev[0],ev[1]))
import pickle
pickle.dump(hl, open("hl.pkl", "wb"))

Epoch 1/100
12/12 [==============================] - 0s 12ms/step - loss: inf - MAE: 462933731421691314176.0000 - val_loss: inf - val_MAE: 967319228821248132775936.0000
Epoch 2/100
12/12 [==============================] - 0s 4ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: nan
Epoch 3/100
12/12 [==============================] - 0s 4ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: nan
Epoch 4/100
12/12 [==============================] - 0s 5ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: nan
Epoch 5/100
12/12 [==============================] - 0s 3ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: nan
Epoch 6/100
12/12 [==============================] - 0s 5ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: nan
Epoch 7/100
12/12 [==============================] - 0s 5ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: nan
Epoch 8/100
12/12 [==============================] - 0s 3ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: na

In [10]:
nn = []
for x in [5,25,125]:
    model = build_model(1,x,"sgd",10e-5)
    tensorboard_cb = tf.keras.callbacks.TensorBoard(get_run_logdir("nn",x))
    model.fit(X_train, y_train, epochs=100,  callbacks=[tensorboard_cb,es], validation_split=0.1)
    ev = model.evaluate(X_test, y_test)
    nn.append((x,ev[0],ev[1]))
import pickle
pickle.dump(nn, open("nn.pkl", "wb")) 

Epoch 1/100
12/12 [==============================] - 1s 16ms/step - loss: 14374.3311 - MAE: 54.7305 - val_loss: 500.3337 - val_MAE: 21.3929
Epoch 2/100
12/12 [==============================] - 0s 5ms/step - loss: 588.5240 - MAE: 22.3446 - val_loss: 498.0657 - val_MAE: 21.3398
Epoch 3/100
12/12 [==============================] - 0s 3ms/step - loss: 586.1573 - MAE: 22.2912 - val_loss: 495.8081 - val_MAE: 21.2868
Epoch 4/100
12/12 [==============================] - 0s 4ms/step - loss: 583.8007 - MAE: 22.2382 - val_loss: 493.5380 - val_MAE: 21.2334
Epoch 5/100
12/12 [==============================] - 0s 4ms/step - loss: 581.4316 - MAE: 22.1849 - val_loss: 491.2727 - val_MAE: 21.1800
Epoch 6/100
12/12 [==============================] - 0s 4ms/step - loss: 579.0676 - MAE: 22.1309 - val_loss: 489.0379 - val_MAE: 21.1272
Epoch 7/100
12/12 [==============================] - 0s 5ms/step - loss: 576.7341 - MAE: 22.0789 - val_loss: 486.7993 - val_MAE: 21.0741
Epoch 8/100
12/12 [===================

In [11]:
opt = []
for x in ["sgd","nesterov","momentum", "adam"]:
    model = build_model(1,25,x,10e-5)
    tensorboard_cb = tf.keras.callbacks.TensorBoard(get_run_logdir("opt",x))
    model.fit(X_train, y_train, epochs=100,  callbacks=[tensorboard_cb,es], validation_split=0.1)
    ev = model.evaluate(X_test, y_test)
    opt.append((x,ev[0],ev[1]))
import pickle
pickle.dump(opt, open("opt.pkl", "wb")) 

Epoch 1/100
12/12 [==============================] - 0s 13ms/step - loss: 427640032.0000 - MAE: 6068.5122 - val_loss: 50.2310 - val_MAE: 5.8455
Epoch 2/100
12/12 [==============================] - 0s 4ms/step - loss: 68.1092 - MAE: 5.6119 - val_loss: 44.7271 - val_MAE: 5.4800
Epoch 3/100
12/12 [==============================] - 0s 5ms/step - loss: 84.9172 - MAE: 6.5169 - val_loss: 63.2436 - val_MAE: 6.5680
Epoch 4/100
12/12 [==============================] - 0s 5ms/step - loss: 84.1590 - MAE: 6.3033 - val_loss: 24.1667 - val_MAE: 3.9273
Epoch 5/100
12/12 [==============================] - 0s 4ms/step - loss: 72.4589 - MAE: 5.7591 - val_loss: 20.9668 - val_MAE: 3.7882
Epoch 6/100
12/12 [==============================] - 0s 4ms/step - loss: 84.2368 - MAE: 6.3764 - val_loss: 23.6464 - val_MAE: 3.8395
Epoch 7/100
12/12 [==============================] - 0s 6ms/step - loss: 74.2032 - MAE: 5.9708 - val_loss: 57.0141 - val_MAE: 6.1931
Epoch 8/100
12/12 [==============================] - 0s 6m

In [ ]:
mom = []
for x in [0.1,0.5,0.9]:
    model = build_model(1,25,"sgd",10e-5,x)
    tensorboard_cb = tf.keras.callbacks.TensorBoard(get_run_logdir("mom",x))
    model.fit(X_train, y_train, epochs=100,  callbacks=[tensorboard_cb,es], validation_split=0.1)
    ev = model.evaluate(X_test, y_test)
    opt.append((x,ev[0],ev[1]))
import pickle
pickle.dump(mom, open("mom.pkl", "wb")) 

In [14]:
param_distribs = {
  "model__n_hidden": [0,1,2,3],
  "model__n_neurons": [5,25,125],
  "model__learning_rate": [10e-6,10e-5,10e-4],
  "model__optimizer": ["sgd","nesterov","momentum", "adam"],
  "model__momentum": [0.1,0.5,0.9]
}


In [16]:
import scikeras
from scikeras.wrappers import KerasRegressor
es = tf.keras.callbacks.EarlyStopping(patience=10, min_delta=1.0, verbose=1)
keras_reg = KerasRegressor(build_model, callbacks=[es])

In [17]:
from sklearn.model_selection import RandomizedSearchCV
rnd_search_cv = RandomizedSearchCV(keras_reg,
param_distribs,
n_iter=30,
cv=3,
verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=100, validation_split=0.1)


Fitting 3 folds for each of 30 candidates, totalling 90 fits
Epoch 1/100
8/8 [==============================] - 1s 32ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: nan
Epoch 2/100
8/8 [==============================] - 0s 5ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: nan
Epoch 3/100
8/8 [==============================] - 0s 5ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: nan
Epoch 4/100
8/8 [==============================] - 0s 5ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: nan
Epoch 5/100
8/8 [==============================] - 0s 5ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: nan
Epoch 6/100
8/8 [==============================] - 0s 5ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: nan
Epoch 7/100
8/8 [==============================] - 0s 5ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: nan
Epoch 8/100
8/8 [==============================] - 0s 5ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: nan
Ep

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 1100, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 1697, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 790, in r2_score
    y_true, y_pred, multioutput
  File "/

8/8 [==============================] - 1s 23ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: nan
Epoch 2/100
8/8 [==============================] - 0s 6ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: nan
Epoch 3/100
8/8 [==============================] - 0s 7ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: nan
Epoch 4/100
8/8 [==============================] - 0s 7ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: nan
Epoch 5/100
8/8 [==============================] - 0s 10ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: nan
Epoch 6/100
8/8 [==============================] - 0s 7ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: nan
Epoch 7/100
8/8 [==============================] - 0s 7ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: nan
Epoch 8/100
8/8 [==============================] - 0s 5ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: nan
Epoch 9/100
8/8 [==============================] - 0s 4ms/step - loss: nan

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 1100, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 1697, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 790, in r2_score
    y_true, y_pred, multioutput
  File "/

8/8 [==============================] - 1s 24ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: nan
Epoch 2/100
8/8 [==============================] - 0s 6ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: nan
Epoch 3/100
8/8 [==============================] - 0s 6ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: nan
Epoch 4/100
8/8 [==============================] - 0s 6ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: nan
Epoch 5/100
8/8 [==============================] - 0s 7ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: nan
Epoch 6/100
8/8 [==============================] - 0s 5ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: nan
Epoch 7/100
8/8 [==============================] - 0s 5ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: nan
Epoch 8/100
8/8 [==============================] - 0s 7ms/step - loss: nan - MAE: nan - val_loss: nan - val_MAE: nan
Epoch 9/100
8/8 [==============================] - 0s 5ms/step - loss: nan 

...

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 1100, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 1697, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 790, in r2_score
    y_true, y_pred, multioutput
  File "/

8/8 [==============================] - 0s 22ms/step - loss: 61161627080196096.0000 - MAE: 109609832.0000 - val_loss: 1190103482368.0000 - val_MAE: 1090918.6250
Epoch 2/100
8/8 [==============================] - 0s 5ms/step - loss: 1172853882880.0000 - MAE: 1082970.8750 - val_loss: 1148506996736.0000 - val_MAE: 1071684.1250
Epoch 3/100
8/8 [==============================] - 0s 9ms/step - loss: 1131834900480.0000 - MAE: 1063864.6250 - val_loss: 1108335001600.0000 - val_MAE: 1052775.0000
Epoch 4/100
8/8 [==============================] - 0s 6ms/step - loss: 1092246110208.0000 - MAE: 1045093.4375 - val_loss: 1069567901696.0000 - val_MAE: 1034199.1250
Epoch 5/100
8/8 [==============================] - 0s 7ms/step - loss: 1054041833472.0000 - MAE: 1026653.2500 - val_loss: 1032156807168.0000 - val_MAE: 1015951.1875
Epoch 6/100
8/8 [==============================] - 0s 6ms/step - loss: 1017174097920.0000 - MAE: 1008538.4375 - val_loss: 996054597632.0000 - val_MAE: 998025.3125
Epoch 7/100
8/8 [

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the test scores are non-finite: [            nan -1.61242174e+01 -3.65112926e+00 -1.55598281e+00
             nan  3.83550266e-01             nan             nan
 -2.17883474e+02             nan             nan  1.15425323e-01
 -1.72328740e+02             nan -5.20974478e+00             nan
             nan -1.77950869e+00             nan             nan
             nan             nan             nan -3.34794787e+02
 -3.90162278e-01  4.74257241e-01 -1.10358453e+01 -1.61519724e+00
             nan             nan]
  category=UserWarning,


12/12 [==============================] - 1s 16ms/step - loss: 190.7958 - MAE: 10.9464 - val_loss: 126.3142 - val_MAE: 10.1553
Epoch 2/100
12/12 [==============================] - 0s 6ms/step - loss: 110.0656 - MAE: 8.0743 - val_loss: 35.3425 - val_MAE: 4.9979
Epoch 3/100
12/12 [==============================] - 0s 6ms/step - loss: 91.9119 - MAE: 7.0951 - val_loss: 32.1051 - val_MAE: 4.6004
Epoch 4/100
12/12 [==============================] - 0s 5ms/step - loss: 75.5544 - MAE: 5.9281 - val_loss: 24.1520 - val_MAE: 3.9960
Epoch 5/100
12/12 [==============================] - 0s 6ms/step - loss: 65.2266 - MAE: 5.6695 - val_loss: 25.4245 - val_MAE: 4.2178
Epoch 6/100
12/12 [==============================] - 0s 5ms/step - loss: 59.7797 - MAE: 5.4925 - val_loss: 37.4658 - val_MAE: 5.0029
Epoch 7/100
12/12 [==============================] - 0s 6ms/step - loss: 63.4842 - MAE: 5.6391 - val_loss: 19.3956 - val_MAE: 3.6097
Epoch 8/100
12/12 [==============================] - 0s 5ms/step - loss: 64

RandomizedSearchCV(cv=3,
                   estimator=KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7fdb6c6dc790>], model=<function build_model at 0x7fdb6cbbaef0>),
                   n_iter=30,
                   param_distributions={'model__learning_rate': [1e-05, 0.0001,
                                                                 0.001],
                                        'model__momentum': [0.1, 0.5, 0.9],
                                        'model__n_hidden': [0, 1, 2, 3],
                                        'model__n_neurons': [5, 25, 125],
                                        'model__optimizer': ['sgd', 'nesterov',
                                                             'momentum',
                                                             'adam']},
                   verbose=2)

In [19]:
rnd_search_cv.best_params_

{'model__learning_rate': 0.001,
 'model__momentum': 0.1,
 'model__n_hidden': 3,
 'model__n_neurons': 125,
 'model__optimizer': 'adam'}

In [ ]:
pickle.dump(rnd_search_cv.best_params_, open("rnd_search.pkl", "wb"))